In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
DATA_ROOT = "/content/drive/MyDrive/Diplomski projekt dataset"

In [ ]:
%pip install -U ultralytics open3d plotly opencv-python

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import minimize
import plotly.graph_objects as go


In [ ]:
img_dir = Path(DATA_ROOT) / "image_2"
lidar_dir = Path(DATA_ROOT) / "lidar pcd"

imgs = sorted(img_dir.glob("*.png"))
bins = sorted(lidar_dir.glob("*.bin"))

print("Images:", imgs[-1])
print("LiDAR bins:", len(bins))
print("First few:", bins[-1])


In [ ]:
def load_kitti_bin(bin_path):
    arr = np.fromfile(str(bin_path), dtype=np.float32)
    arr = arr.reshape(-1, 4)  # x,y,z,intensity
    return arr

def show_points_plotly(pts_xyz, title="LiDAR points", max_points=80000):
    pts = np.asarray(pts_xyz)
    if pts.shape[0] > max_points:
        idx = np.random.choice(pts.shape[0], max_points, replace=False)
        pts = pts[idx]

    fig = go.Figure(data=[
        go.Scatter3d(
            x=pts[:,0], y=pts[:,1], z=pts[:,2],
            mode="markers",
            marker=dict(size=1, opacity=0.6),
        )
    ])
    fig.update_layout(title=title, scene=dict(aspectmode="data"), height=700)
    fig.show()


In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2

yolo = YOLO("yolov8n-seg.pt")

def segment_boat_instances(img_bgr):
    """
    Vrati listu maski (uint8 0/255) za SVAKI brod (instanca),
    usklađeno na rezoluciju originalne slike.
    """
    res = yolo.predict(img_bgr, verbose=False)[0]

    names = res.names
    boat_ids = [i for i, n in names.items() if n == "boat"]

    if not boat_ids or res.masks is None:
        return [], res

    masks = res.masks.data.cpu().numpy()  # (N, Hm, Wm)
    clss  = res.boxes.cls.cpu().numpy().astype(int)

    h, w = img_bgr.shape[:2]
    out = []

    for k in range(len(clss)):
        if clss[k] not in boat_ids:
            continue

        m = (masks[k] * 255).astype(np.uint8)
        if m.shape[:2] != (h, w):
            m = cv2.resize(m, (w, h), interpolation=cv2.INTER_NEAREST)

        out.append(m)

    return out, res
def mask_props(mask_u8):
    ys, xs = np.where(mask_u8 > 0)
    if len(xs) == 0:
        return None

    cx = float(xs.mean())
    cy = float(ys.mean())
    x1, x2 = int(xs.min()), int(xs.max())
    y1, y2 = int(ys.min()), int(ys.max())
    area = int(len(xs))

    return {
        "centroid_xy": (cx, cy),
        "bbox_xyxy": (x1, y1, x2, y2),
        "area_px": area
    }


In [ ]:
import open3d as o3d

def preprocess_points(pts_xyz, voxel=0.15, nb_neighbors=20, std_ratio=2.0):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts_xyz)

    if voxel:
        pcd = pcd.voxel_down_sample(voxel_size=voxel)

    pcd, ind = pcd.remove_statistical_outlier(
        nb_neighbors=nb_neighbors,
        std_ratio=std_ratio
    )
    return pcd
from collections import deque

def euclidean_clustering(pcd, tolerance=0.8, min_cluster_size=30, max_cluster_size=200000):
    pts = np.asarray(pcd.points)
    if len(pts) == 0:
        return []

    kdtree = o3d.geometry.KDTreeFlann(pcd)
    visited = np.zeros(len(pts), dtype=bool)
    clusters = []

    for i in range(len(pts)):
        if visited[i]:
            continue

        queue = deque([i])
        visited[i] = True
        cluster_idx = []

        while queue:
            idx = queue.popleft()
            cluster_idx.append(idx)

            _, idxs, _ = kdtree.search_radius_vector_3d(pcd.points[idx], tolerance)
            for j in idxs:
                if not visited[j]:
                    visited[j] = True
                    queue.append(j)

        if min_cluster_size <= len(cluster_idx) <= max_cluster_size:
            clusters.append(cluster_idx)

    return clusters
def roi_filter(pts_xyz,
               x_min=-75.0, x_max=80.0,
               y_min=-150.0, y_max=0.0,
               ):
    x, y, z = pts_xyz[:,0], pts_xyz[:,1], pts_xyz[:,2]
    m = (
        (x >= x_min) & (x <= x_max) &
        (y >= y_min) & (y <= y_max)

    )
    return pts_xyz[m]
import plotly.graph_objects as go
import numpy as np

def show_clusters_plotly(pcd, clusters, title="Euclidean clusters", max_points=120000):
    pts = np.asarray(pcd.points)
    if len(pts) == 0:
        print("No points.")
        return

    labels = np.full(len(pts), -1, dtype=int)
    for ci, idxs in enumerate(clusters):
        labels[idxs] = ci

    if len(pts) > max_points:
        keep = np.random.choice(len(pts), max_points, replace=False)
        pts = pts[keep]
        labels = labels[keep]

    uniq = np.unique(labels)

    fig = go.Figure()
    for lb in uniq:
        mask = labels == lb
        name = "noise/unclustered" if lb == -1 else f"cluster {lb}"
        fig.add_trace(go.Scatter3d(
            x=pts[mask,0], y=pts[mask,1], z=pts[mask,2],
            mode="markers",
            marker=dict(size=1),
            name=name
        ))

    fig.update_layout(
        title=title,
        scene=dict(aspectmode="data"),
        height=700
    )
    fig.show()


def process_lidar_euclid(bin_path,
                         roi_kwargs=None,
                         voxel=0.15,
                         tolerance=0.8,
                         min_cluster_size=30):
    roi_kwargs = roi_kwargs or {}

    pts = load_kitti_bin(bin_path)
    print(bin_path)
    pts_xyz = pts[:, :3]

    pts_xyz = roi_filter(pts_xyz, **roi_kwargs)

    pcd = preprocess_points(pts_xyz, voxel=voxel)

    clusters = euclidean_clustering(
        pcd,
        tolerance=tolerance,
        min_cluster_size=min_cluster_size
    )

    print("points after preprocess:", np.asarray(pcd.points).shape[0])
    print("clusters found:", len(clusters))

    show_clusters_plotly(pcd, clusters, title=f"Euclidean clusters — {bin_path.name}")

    return pcd, clusters


In [ ]:
def build_frame_data_instances(i,
                               roi_kwargs=None,
                               voxel=0.20,
                               tolerance=1.5,
                               min_cluster_size=50):
    roi_kwargs = roi_kwargs or dict(x_min=-80, x_max=80, y_min=-150, y_max=150)

    img = cv2.imread(str(imgs[i]))
    inst_masks, _ = segment_boat_instances(img)

    masks_list = []
    for mi, m in enumerate(inst_masks):
        props = mask_props(m)
        if props is None:
            continue
        masks_list.append({
            "mask_id": mi,
            "centroid_xy": props["centroid_xy"],
            "bbox_xyxy": props["bbox_xyxy"],
            "area_px": props["area_px"],
            "mask": m.astype(np.uint8)  # <-- DODANO
        })

    pcd, clusters = process_lidar_euclid(
        bins[i],
        roi_kwargs=roi_kwargs,
        voxel=0.20,
        tolerance=5.0,
        min_cluster_size=100
    )

    pts = np.asarray(pcd.points)

    clusters_list = []
    for ci, idxs in enumerate(clusters):
        idxs = np.array(list(idxs), dtype=np.int64)
        cluster_pts = pts[idxs]

        if cluster_pts.shape[0] == 0:
            continue

        centroid = cluster_pts.mean(axis=0)

        clusters_list.append({
            "cluster_id": ci,
            "centroid_xyz": centroid.astype(np.float32),
            "points_xyz": cluster_pts.astype(np.float32),  # <-- DODANO
            "size": int(cluster_pts.shape[0])
        })

    frame_data = {
        "pair_idx": i,
        "image_path": str(imgs[i]),
        "lidar_path": str(bins[i]),
        "masks": masks_list,
        "clusters": clusters_list,
        "img_hw": img.shape[:2]  # <-- korisno za DL rasterizaciju
    }

    return frame_data


In [ ]:
def show_instance_masks(idx):
    fd = build_frame_data_instances(idx)
    img = cv2.imread(fd["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    plt.imshow(img_rgb); plt.title(f"Image {idx}"); plt.axis("off")

    plt.subplot(1,2,2)
    overlay = img_rgb.copy()
    for m in fd["masks"]:
        mask = (m["mask"] > 0)
        overlay[mask] = (0.4*overlay[mask] + 0.6*np.array([255,255,255])).astype(np.uint8)
        cx, cy = m["centroid_xy"]
        plt.text(cx, cy, f'M{m["mask_id"]}', color="yellow", fontsize=12, weight="bold")
    plt.imshow(overlay); plt.title("YOLO boat instance masks"); plt.axis("off")
    plt.show()
show_instance_masks(25)
#for i in range(40, 50):
#    show_instance_masks(i)


In [ ]:
K = np.array([
    [1220.2990146560746, 0., 1016.9903640851892],
    [0., 1209.7297893764865, 562.18789131419339],
    [0., 0., 1.]
], dtype=np.float64)

dist = np.array([
    -0.44921633687928347, 0.20895767078880936,
    -0.0038733225065608686, -0.010605998170363269,
    -0.047375804391877364, 0., 0., 0., 0., 0., 0., 0., 0., 0.
], dtype=np.float64).reshape(-1,1)

def make_A_lidar_to_cam0(mirror_x=False):
    """
    Transform from LiDAR coords (X-left/right, -Y-forward, Z-up)
    to Camera coords (X-right, Y-down, Z-forward)
    """
    return np.array([
        [(-1.0 if mirror_x else 1.0), 0.0,  0.0],
        [0.0,  0.0, -1.0],
        [0.0, -1.0,  0.0],
    ], dtype=np.float64)

    if mirror_x:
        A[0, 0] *= -1.0
    return A
def visualize_axes(points, A):
    import plotly.graph_objects as go
    P = (A @ points.T).T
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=P[:,0], y=P[:,1], z=P[:,2],
        mode='markers', marker=dict(size=2, color=P[:,2], colorscale='Viridis'),
        name='Transformed LiDAR'
    ))
    fig.add_trace(go.Scatter3d(
        x=[0,1], y=[0,0], z=[0,0], mode='lines+text',
        line=dict(color='red', width=5),
        text=["", "X_cam"], textposition="top center"
    ))
    fig.add_trace(go.Scatter3d(
        x=[0,0], y=[0,1], z=[0,0], mode='lines+text',
        line=dict(color='green', width=5),
        text=["", "Y_cam"], textposition="top center"
    ))
    fig.add_trace(go.Scatter3d(
        x=[0,0], y=[0,0], z=[0,1], mode='lines+text',
        line=dict(color='blue', width=5),
        text=["", "Z_cam (forward)"], textposition="top center"
    ))
    fig.update_layout(scene=dict(aspectmode='data'), title="LiDAR→Camera orientation check")
    fig.show()


def lidar_to_cam0(points_xyz, A):
    P = np.asarray(points_xyz, dtype=np.float64)
    return (A @ P.T).T

def project_points(P0, rvec, tvec, K, dist):
    P0 = np.asarray(P0, np.float64).reshape(-1,3)
    rvec = np.asarray(rvec, np.float64).reshape(3,1)
    tvec = np.asarray(tvec, np.float64).reshape(3,1)
    uv, _ = cv2.projectPoints(P0.reshape(-1,1,3), rvec, tvec, K, dist)
    uv = uv.reshape(-1,2)

    R, _ = cv2.Rodrigues(rvec)
    Pc = (R @ P0.T + tvec).T
    Z = Pc[:,2]
    return uv, Z


In [ ]:
def depth_image_from_points(P0, rvec, tvec, K, dist, H, W):
    uv, Z = project_points(P0, rvec, tvec, K, dist)

    u = uv[:,0]
    v = uv[:,1]
    valid = np.isfinite(u) & np.isfinite(v) & (Z > 1e-3) & (u>=0) & (u < W) & (v>=0) & (v < H)

    u_i = u[valid].astype(np.int32)
    v_i = v[valid].astype(np.int32)
    z_v = Z[valid]

    depth = np.full((H,W), np.inf, dtype=np.float64)

    for ui, vi, zi in zip(u_i, v_i, z_v):
        if zi < depth[vi, ui]:
            depth[vi, ui] = zi

    return depth

def show_depth_map(depth, title="Depth map"):
    d = depth.copy()
    finite = np.isfinite(d)
    if finite.sum() == 0:
        print("No finite depth pixels")
        return
    d2 = d[finite]
    vmin, vmax = np.percentile(d2, 5), np.percentile(d2, 95)

    plt.figure(figsize=(12,5))
    plt.imshow(np.clip(d, vmin, vmax))
    plt.title(title)
    plt.axis("off")
    plt.colorbar()
    plt.show()


In [ ]:
def image_edges(img_bgr, low=80, high=160):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, low, high)
    return edges

def depth_edges(depth, grad_thr=0.5):
    d = depth.copy()
    d[~np.isfinite(d)] = 0.0

    gx = cv2.Sobel(d, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(d, cv2.CV_64F, 0, 1, ksize=3)
    g = np.sqrt(gx*gx + gy*gy)

    m = np.percentile(g[g>0], 95) if np.any(g>0) else 1.0
    g_n = g / (m + 1e-9)

    edges = (g_n > grad_thr).astype(np.uint8) * 255
    return edges, g_n

def show_edges(img_bgr, e_img, e_depth, title="Edges"):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(16,5))

    plt.subplot(1,3,1); plt.imshow(img_rgb); plt.title("Image"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(e_img, cmap="gray"); plt.title("Image edges (Canny)"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(e_depth, cmap="gray"); plt.title("Depth edges (from LiDAR)"); plt.axis("off")

    plt.suptitle(title)
    plt.show()

def overlay_edges(img_bgr, e_img, e_depth, title="Overlay"):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB).copy()
    img_rgb[e_img>0]   = [0,255,0]
    img_rgb[e_depth>0] = [255,0,0]

    plt.figure(figsize=(14,7))
    plt.imshow(img_rgb)
    plt.title(title + " (green=image edges, red=lidar depth edges)")
    plt.axis("off")
    plt.show()


In [ ]:
def distance_transform_of_edges(edge_u8):
    inv = (edge_u8 == 0).astype(np.uint8)
    dt = cv2.distanceTransform(inv, cv2.DIST_L2, 5).astype(np.float64)
    return dt

def edge_alignment_cost(dt_img, e_depth, min_edge_pixels=200):
    ys, xs = np.where(e_depth > 0)
    if len(xs) < min_edge_pixels:
        return 1e6, {"depth_edge_pixels": len(xs), "mean_dt": None}

    vals = dt_img[ys, xs]
    return float(np.mean(vals)), {"depth_edge_pixels": len(xs), "mean_dt": float(np.mean(vals))}


In [ ]:
def eval_and_plot_one(frame_idx, rvec, tvec, mirror_x=True,
                      canny_low=80, canny_high=160,
                      depth_grad_thr=0.35):
    fd = build_frame_data_instances(frame_idx)
    img = cv2.imread(fd["image_path"])
    H, W = img.shape[:2]

    pts = load_kitti_bin(fd["lidar_path"])[:, :3]

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    P0 = lidar_to_cam0(pts, A)

    depth = depth_image_from_points(P0, rvec, tvec, K, dist, H, W)
    show_depth_map(depth, title=f"Depth map (frame {frame_idx})")

    e_img = image_edges(img, low=canny_low, high=canny_high)
    dt_img = distance_transform_of_edges(e_img)

    e_depth, g_n = depth_edges(depth, grad_thr=depth_grad_thr)

    show_edges(img, e_img, e_depth, title=f"Frame {frame_idx}")
    overlay_edges(img, e_img, e_depth, title=f"Frame {frame_idx}")

    c, info = edge_alignment_cost(dt_img, e_depth)
    print("Cost:", c)
    print("Info:", info)

    return c, info

rvec0 = np.zeros(3)
tvec0 = np.zeros(3)

c0, info0 = eval_and_plot_one(frame_idx=40, rvec=rvec0, tvec=tvec0, mirror_x=True)


In [ ]:
  def rvec_from_yaw_deg(yaw_deg):
      yaw = np.deg2rad(yaw_deg)
      R = np.array([
          [ np.cos(yaw), 0, np.sin(yaw)],
          [ 0,           1, 0          ],
          [-np.sin(yaw), 0, np.cos(yaw)]
      ], dtype=np.float64)
      rvec, _ = cv2.Rodrigues(R)
      return rvec.ravel()

  def coarse_search_yaw(frame_idx, yaw_list_deg, tvec=np.zeros(3), mirror_x=True,
                        canny_low=80, canny_high=160, depth_grad_thr=0.35):
      fd = build_frame_data_instances(frame_idx)
      img = cv2.imread(fd["image_path"])
      H, W = img.shape[:2]
      pts = load_kitti_bin(fd["lidar_path"])[:, :3]

      A = make_A_lidar_to_cam0(mirror_x=mirror_x)
      P0 = lidar_to_cam0(pts, A)

      e_img = image_edges(img, low=canny_low, high=canny_high)
      dt_img = distance_transform_of_edges(e_img)

      costs = []
      for yaw in yaw_list_deg:
          rvec = rvec_from_yaw_deg(yaw)
          depth = depth_image_from_points(P0, rvec, tvec, K, dist, H, W)
          e_depth, _ = depth_edges(depth, grad_thr=depth_grad_thr)
          c, info = edge_alignment_cost(dt_img, e_depth)
          costs.append((yaw, c, info["depth_edge_pixels"]))

      ys = [a for a,_,_ in costs]
      cs = [b for _,b,_ in costs]
      plt.figure(figsize=(10,4))
      plt.plot(ys, cs, marker="o")
      plt.title(f"Coarse yaw search — frame {frame_idx}")
      plt.xlabel("yaw (deg)")
      plt.ylabel("cost (mean DT)")
      plt.grid(True, alpha=0.3)
      plt.show()

      costs_sorted = sorted(costs, key=lambda x: x[1])
      print("Top 5 yaw candidates:")
      for row in costs_sorted[:5]:
          print(row)

      top5 = costs_sorted[:5]

      best_yaw = top5[0][0]
      best_rvec = rvec_from_yaw_deg(best_yaw)
      print("best rvec", best_rvec)

      for i, (yaw, c, npx) in enumerate(top5, 1):
          rvec = rvec_from_yaw_deg(yaw)
          print(f"[{i}/5] yaw={yaw} cost={c} depth_edge_pixels={npx}")
          eval_and_plot_one(frame_idx, rvec, tvec, mirror_x=mirror_x,
                            canny_low=canny_low, canny_high=canny_high, depth_grad_thr=depth_grad_thr)


      return best_rvec, tvec, costs_sorted

  yaw_list = list(range(0, 359, 5))
  #yaw_list = list(range(-20, 21, 1))
  best_rvec26, best_tvec26, costs_sorted = coarse_search_yaw(frame_idx=26, yaw_list_deg=yaw_list, mirror_x=True)

  best_rvec40, best_tvec40, costs_sorted = coarse_search_yaw(frame_idx=40, yaw_list_deg=yaw_list, mirror_x=True)

  best_rvec25, best_tvec25, costs_sorted2 = coarse_search_yaw(frame_idx=25, yaw_list_deg=yaw_list, mirror_x=True)
  best_rvec50, best_tvec50, costs_sorted2 = coarse_search_yaw(frame_idx=50, yaw_list_deg=yaw_list, mirror_x=True)
  best_rvec55, best_tvec55, costs_sorted2 = coarse_search_yaw(frame_idx=55, yaw_list_deg=yaw_list, mirror_x=True)

In [ ]:
from dataclasses import dataclass

@dataclass
class FrameCache:
    fi: int
    img: np.ndarray
    H: int
    W: int
    P0: np.ndarray
    dt_img: np.ndarray
    e_img: np.ndarray

def _gather_cluster_points(frame_data, max_points=None):
    pts_list = []
    for c in frame_data["clusters"]:
        pts_list.append(c["points_xyz"])
    if len(pts_list) == 0:
        return np.zeros((0,3), dtype=np.float32)
    pts = np.concatenate(pts_list, axis=0).astype(np.float32)

    if (max_points is not None) and (pts.shape[0] > max_points):
        idx = np.random.choice(pts.shape[0], max_points, replace=False)
        pts = pts[idx]
    return pts

def build_cache_from_frame_data(frame_indices,
                               mirror_x=True,
                               canny_low=80, canny_high=160,
                               use_clusters=True,
                               max_lidar_points=120_000,
                               roi_kwargs=None,
                               voxel=0.20,
                               tolerance=5.0,
                               min_cluster_size=100):

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    caches = []
    frame_datas = []

    for fi in frame_indices:
        fd = build_frame_data_instances(
            fi,
            roi_kwargs=roi_kwargs,
            voxel=voxel,
            tolerance=tolerance,
            min_cluster_size=min_cluster_size
        )
        frame_datas.append(fd)

        img = cv2.imread(fd["image_path"])
        H, W = img.shape[:2]

        e_img = image_edges(img, low=canny_low, high=canny_high)
        dt_img = distance_transform_of_edges(e_img)

        if use_clusters:
            pts = _gather_cluster_points(fd, max_points=max_lidar_points)
        else:
            # fallback: sve točke iz bina
            pts = load_kitti_bin(fd["lidar_path"])[:, :3].astype(np.float32)
            if pts.shape[0] > max_lidar_points:
                idx = np.random.choice(pts.shape[0], max_lidar_points, replace=False)
                pts = pts[idx]

        P0 = lidar_to_cam0(pts, A)  # precompute

        caches.append(FrameCache(fi=fi, img=img, H=H, W=W, P0=P0, dt_img=dt_img, e_img=e_img))

    return caches, frame_datas


In [ ]:
def _contour_score(cnt, B):
    if cnt is None or np.asarray(cnt).shape[0] < 3:
        return -1e18

    area = float(cv2.contourArea(cnt.astype(np.float32)))
    if area <= 1e-6:
        return -1e18

    ncc, _ = cv2.connectedComponents((B > 0).astype(np.uint8))
    fragments = int(ncc - 1)

    frag_pen = 2000.0 * max(0, fragments - 1)

    xs = cnt[:,0]; ys = cnt[:,1]
    w = float(xs.max() - xs.min() + 1e-9)
    h = float(ys.max() - ys.min() + 1e-9)
    box_area = w * h
    fill_ratio = area / (box_area + 1e-12)
    thin_pen = 1000.0 * max(0.0, 0.08 - fill_ratio)

    return area - frag_pen - thin_pen
def orient_plane_by_rule(points_xyz, P2, plane, want="Y_left"):

    P = np.asarray(points_xyz, np.float64)
    u = np.asarray(P2[:,0], np.float64)

    if want == "Y_left":
        y = P[:,1]
        corr = float(np.mean((u - u.mean()) * (y - y.mean())))
        if corr > 0:
            P2 = P2.copy()
            P2[:,0] *= -1.0
            flipped = True
        else:
            flipped = False
        return P2, flipped, corr

    raise ValueError("unknown want rule")


def choose_best_plane_by_raster(points_xyz, mirror_x=True, mirror_z=False,
                                grid=256, splat_r=2, close_r=4, verbose=False):

    P = np.asarray(points_xyz, np.float64)
    if P.size == 0:
        return "xz", False

    candidates = []
    for plane in ("xy", "xz", "yz"):
        for flip_second in (False, True):
            P2 = glue_to_plane(P, plane=plane, mirror_x=mirror_x, mirror_y=False, mirror_z=mirror_z)
            if flip_second:
                P2 = P2.copy()
                P2[:,1] *= -1.0

            cnt, B = contour_from_points_raster(P2, grid=grid, splat_r=splat_r, close_r=close_r)
            score = _contour_score(cnt, B)
            candidates.append((score, plane, flip_second))

    best = max(candidates, key=lambda t: t[0])
    if verbose:
        print("plane candidates (score, plane, flip2):")
        for s, pl, fl in sorted(candidates, reverse=True)[:6]:
            print(f"  {s:10.1f}  {pl}  flip2={fl}")
        print("chosen:", best[1], "flip2=", best[2], "score=", best[0])

    return best[1], best[2]
def debug_one_cluster_shape(fd, cluster_id, mirror_x=True, mirror_z=False,
                           grid=256, splat_r=2, close_r=4):
    c = next((cc for cc in fd["clusters"] if cc["cluster_id"] == cluster_id), None)
    if c is None:
        print("Nema tog cluster_id")
        return

    pts = np.asarray(c["points_xyz"], np.float64)

    plane, flip2 = choose_best_plane_by_raster(pts, mirror_x=mirror_x, mirror_z=mirror_z,
                                           grid=grid, splat_r=splat_r, close_r=close_r, verbose=True)

    P2 = glue_to_plane(pts, plane=plane, mirror_x=mirror_x, mirror_y=False, mirror_z=mirror_z)

    if flip2:
        P2 = P2.copy()
        P2[:,1] *= -1.0

    P2, flip_u, corr = orient_plane_by_rule(pts, P2, plane=plane, want="Y_left")
    print(f"[DEBUG] C{cluster_id}: chosen={plane.upper()} flip2={flip2} flip_u={flip_u} corr(u,Y)={corr:.3f}")

    cnt, B = contour_from_points_raster(P2, grid=grid, splat_r=splat_r, close_r=close_r)

    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.scatter(P2[:,0], P2[:,1], s=3)
    plt.gca().set_aspect("equal","box")
    plt.title(f"raw {plane.upper()} cluster {cluster_id} (flip2={flip2})")
    plt.grid(True, alpha=0.2)

    plt.subplot(1,3,2)
    plt.imshow(B, cmap="gray"); plt.title("raster"); plt.axis("off")

    plt.subplot(1,3,3)
    plt.imshow(B, cmap="gray")
    if cnt.shape[0] > 0:
        plt.plot(cnt[:,0], cnt[:,1], linewidth=2)
    plt.title("contour"); plt.axis("off")
    plt.tight_layout(); plt.show()

fd = build_frame_data_instances(25)

#debug_one_cluster_shape(fd, cluster_id=4, mirror_x=True, mirror_z=False, grid=256, splat_r=2, close_r=4)

In [ ]:
import numpy as np
import cv2
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.lines import Line2D


def filter_fd_min_points(fd, min_cluster_points=200):
    fd2 = dict(fd)
    fd2["clusters"] = [
        c for c in fd["clusters"]
        if int(np.asarray(c["points_xyz"]).shape[0]) >= int(min_cluster_points)
    ]
    return fd2


def largest_contour(mask_u8):
    mk = mask_u8.astype(np.uint8)
    if mk.max() <= 1:
        mk = mk * 255
    cnts, _ = cv2.findContours(mk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        return None
    cnt = max(cnts, key=cv2.contourArea)
    return cnt[:, 0, :]  # (M,2)

def resample_polyline(points_xy, n=80):
    P = np.asarray(points_xy, np.float64)
    if P.shape[0] < 2:
        return np.zeros((n,2), np.float64)
    if not np.allclose(P[0], P[-1]):
        P = np.vstack([P, P[0]])
    seg = np.linalg.norm(P[1:] - P[:-1], axis=1)
    s = np.concatenate([[0.0], np.cumsum(seg)])
    total = s[-1]
    if total < 1e-9:
        return np.tile(P[0], (n,1))
    t = np.linspace(0, total, n, endpoint=False)
    out = np.zeros((n,2), np.float64)
    j = 0
    for i, ti in enumerate(t):
        while j < len(s)-2 and s[j+1] < ti:
            j += 1
        t0, t1 = s[j], s[j+1]
        a = 0.0 if (t1-t0) < 1e-12 else (ti - t0) / (t1 - t0)
        out[i] = (1-a)*P[j] + a*P[j+1]
    return out

def points_from_mask(mask_u8, n=80):
    cnt = largest_contour(mask_u8)
    if cnt is None:
        return np.zeros((n,2), np.float64)
    return resample_polyline(cnt, n=n)


def choose_best_plane(points_xyz, q_lo=0.05, q_hi=0.95, eps=1e-9):
    """
    Odaberi ravninu XY/XZ/YZ po tome koje dvije osi imaju najveći raspon.
    Vraća: plane_str, spans (x_span, y_span, z_span)
    """
    P = np.asarray(points_xyz, np.float64)
    if P.size == 0:
        return "xz", (0.0, 0.0, 0.0)

    lo = np.quantile(P, q_lo, axis=0)
    hi = np.quantile(P, q_hi, axis=0)
    spans = (hi - lo) + eps  # (sx, sy, sz)

    order = np.argsort(spans)[::-1]
    top2 = set(order[:2])

    if top2 == {0, 1}: plane = "xy"
    elif top2 == {0, 2}: plane = "xz"
    else: plane = "yz"

    return plane, tuple(float(s) for s in spans)

def glue_to_plane(points_xyz, plane="xz",
                  mirror_x=True, mirror_y=False, mirror_z=False):
    """
    Pretvori 3D točke u 2D ovisno o odabranoj ravnini
    """
    P = np.asarray(points_xyz, np.float64)
    x = P[:,0].copy()
    y = P[:,1].copy()
    z = P[:,2].copy()
    if mirror_x: x = -x
    if mirror_y: y = -y
    if mirror_z: z = -z

    if plane == "xy":
        return np.stack([x, y], axis=1)
    if plane == "xz":
        return np.stack([x, z], axis=1)
    if plane == "yz":
        return np.stack([y, z], axis=1)
    raise ValueError("plane mora biti 'xy', 'xz' ili 'yz'")

def normalize_unit_box(P2, eps=1e-9):
    P = np.asarray(P2, np.float64)
    if P.shape[0] == 0:
        return P
    mn = P.min(axis=0)
    mx = P.max(axis=0)
    span = mx - mn
    s = max(span[0], span[1]) + eps
    return (P - mn) / s

def contour_from_points_raster(P2, grid=256, splat_r=2, close_r=6):
    """
    From 2D points -> binary image -> dilate/close -> largest contour -> (N,2) in grid coords.
    """
    P = np.asarray(P2, np.float64)
    if P.shape[0] == 0:
        return np.zeros((0,2), np.float64), np.zeros((grid,grid), np.uint8)

    U = normalize_unit_box(P)
    x = U[:,0]; y = U[:,1]
    lo_x, hi_x = np.quantile(x, [0.01, 0.99])
    lo_y, hi_y = np.quantile(y, [0.01, 0.99])
    keep = (x >= lo_x) & (x <= hi_x) & (y >= lo_y) & (y <= hi_y)
    U2 = U[keep]

    u = np.clip((U2[:,0]*(grid-1)).round().astype(np.int32), 0, grid-1)
    v = np.clip((U2[:,1]*(grid-1)).round().astype(np.int32), 0, grid-1)
    B = np.zeros((grid, grid), np.uint8)
    B[(grid-1) - v, u] = 1   # <-- ovo je "Z gore"


    k = 2*splat_r + 1
    ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k,k))
    B = cv2.dilate(B, ker, iterations=1)
    open_r = 1
    if open_r > 0:
        ko = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*open_r+1, 2*open_r+1))
        B = cv2.morphologyEx(B, cv2.MORPH_OPEN, ko, iterations=1)

    if close_r > 0:
        kc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*close_r+1, 2*close_r+1))
        B = cv2.morphologyEx(B, cv2.MORPH_CLOSE, kc, iterations=1)

    cnts, _ = cv2.findContours((B*255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        return np.zeros((0,2), np.float64), B

    cnt = max(cnts, key=cv2.contourArea)[:,0,:].astype(np.float64)
    return cnt, B
def normalize_zero_mean_unit(P, eps=1e-9):
    P = np.asarray(P, np.float64)
    if P.shape[0] == 0:
        return P
    Q = P - P.mean(axis=0, keepdims=True)
    rms = np.sqrt(np.mean(np.sum(Q**2, axis=1))) + eps
    return Q / rms

def rotate_points(P, ang_rad):
    ca, sa = np.cos(ang_rad), np.sin(ang_rad)
    R = np.array([[ca, -sa],[sa, ca]], np.float64)
    return P @ R.T
def aspect_ratio(P):
    P = np.asarray(P, np.float64)
    if P.shape[0] == 0:
        return 1.0
    w = (P[:,0].max() - P[:,0].min()) + 1e-9
    h = (P[:,1].max() - P[:,1].min()) + 1e-9
    return float(h / w)

def shape_context_distance_rot(P, Q, n=80, n_rot=12):

    P = resample_polyline(P, n=n) if np.asarray(P).shape[0] >= 2 else np.zeros((n,2))
    Q = resample_polyline(Q, n=n) if np.asarray(Q).shape[0] >= 2 else np.zeros((n,2))

    Pn = normalize_zero_mean_unit(P)
    Qn = normalize_zero_mean_unit(Q)

    best = np.inf
    for k in range(n_rot):
        ang = 2*np.pi * k / n_rot
        Qr = rotate_points(Qn, ang)

        dP = shape_context(Pn)
        dQ = shape_context(Qr)
        C = chi2_cost_matrix(dP, dQ)
        ri, cj = linear_sum_assignment(C)
        cost = float(np.mean(C[ri, cj])) if len(ri) else np.inf
        if cost < best:
            best = cost

    return best


def chi2_cost_matrix(descA, descB, eps=1e-10):
    A = descA[:,None,:]
    B = descB[None,:,:]
    num = (A - B)**2
    den = (A + B + eps)
    return 0.5 * np.sum(num / den, axis=2)
def shape_context(points_xy, nbins_r=5, nbins_theta=12, r_inner=0.125, r_outer=2.0):
    P = np.asarray(points_xy, np.float64)
    N = P.shape[0]
    if N == 0:
        return np.zeros((0, nbins_r*nbins_theta), np.float64)

    dxy = P[None,:,:] - P[:,None,:]
    r = np.linalg.norm(dxy, axis=2)
    mean_r = np.mean(r[r > 1e-9]) if np.any(r > 1e-9) else 1.0
    r = r / (mean_r + 1e-12)
    theta = np.arctan2(dxy[:,:,1], dxy[:,:,0])

    r_bins = np.logspace(np.log10(r_inner), np.log10(r_outer), nbins_r+1)
    t_bins = np.linspace(-np.pi, np.pi, nbins_theta+1)

    desc = np.zeros((N, nbins_r, nbins_theta), np.float64)
    for i in range(N):
        ri = r[i]
        ti = theta[i]
        rb = np.digitize(ri, r_bins) - 1
        tb = np.digitize(ti, t_bins) - 1
        valid = (rb >= 0) & (rb < nbins_r) & (tb >= 0) & (tb < nbins_theta)
        valid[i] = False
        rbv = rb[valid]; tbv = tb[valid]
        for a, b in zip(rbv, tbv):
            desc[i, a, b] += 1.0
        s = desc[i].sum()
        if s > 1e-9:
            desc[i] /= s
    return desc.reshape(N, -1)

def chi2_cost_matrix(descA, descB, eps=1e-10):
    A = descA[:,None,:]
    B = descB[None,:,:]
    num = (A - B)**2
    den = (A + B + eps)
    return 0.5 * np.sum(num / den, axis=2)
def shape_context_distance(P, Q, n=80):
    P = resample_polyline(P, n=n) if np.asarray(P).shape[0] >= 2 else np.zeros((n,2))
    Q = resample_polyline(Q, n=n) if np.asarray(Q).shape[0] >= 2 else np.zeros((n,2))
    dP = shape_context(P)
    dQ = shape_context(Q)
    C = chi2_cost_matrix(dP, dQ)
    ri, cj = linear_sum_assignment(C)
    return float(np.mean(C[ri, cj]))

def cluster_contour_auto(
    pts_xyz,
    mirror_x=True, mirror_z=False,
    grid=256, splat_r=2, close_r=4,
    want="Y_left",
    verbose=False,
    cluster_id=None,
    return_points=False
):
    plane, flip2 = choose_best_plane_by_raster(
        pts_xyz, mirror_x=mirror_x, mirror_z=mirror_z,
        grid=grid, splat_r=splat_r, close_r=close_r,
        verbose=False
    )

    P2 = glue_to_plane(pts_xyz, plane=plane, mirror_x=mirror_x, mirror_y=False, mirror_z=mirror_z)

    if flip2:
        P2 = P2.copy()
        P2[:,1] *= -1.0

    flip_u = False
    corr = 0.0
    if want == "Y_left" and plane == "yz":
        P2, flip_u, corr = orient_plane_by_rule(pts_xyz, P2, plane=plane, want=want)

    cnt, B = contour_from_points_raster(P2, grid=grid, splat_r=splat_r, close_r=close_r)

    info = {"plane": plane, "flip2": flip2, "flip_u": flip_u, "corr": corr}
    if verbose and (cluster_id is not None):
        print(f"C{cluster_id}: plane={plane} flip2={flip2} flip_u={flip_u} corr={corr:.3f} | cntN={cnt.shape[0]}")

    if return_points:
        return cnt, info, P2
    return cnt, info


def match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True, mirror_z=False,
    n_samples=80,
    min_cluster_points=200,
    only_y_lt_0=False,
    grid=256, splat_r=2, close_r=4,
    n_rot=12,
    projector=None,
    verbose=True
):


    # ---- 1) filter clusters by min points ----
    fd_f = filter_fd_min_points(fd, min_cluster_points=min_cluster_points)

    masks = fd_f.get("masks", [])
    clusters_all = fd_f.get("clusters", [])

    if not masks or not clusters_all:
        if verbose:
            print(f"No masks or clusters after min_points filter. masks={len(masks)} clusters={len(clusters_all)}")
        return [], None, fd_f

    # ---- 2) filter clusters by y<0 if needed ----
    clusters = []
    for c in clusters_all:
        cy = float(np.asarray(c["centroid_xyz"], np.float64)[1])
        if only_y_lt_0 and not (cy < 0):
            continue
        clusters.append(c)

    fd2 = dict(fd_f)
    fd2["clusters"] = clusters  # important for plotting consistency

    if not clusters:
        if verbose:
            print("No clusters left after y<0 filter.")
        return [], None, fd2

    # ---- 3) mask boundary points (always in image coords) ----
    mask_pts = []
    valid_mask = []
    for m in masks:
        Pm = points_from_mask(m["mask"], n=n_samples)
        ok = np.asarray(Pm).shape[0] >= 3 and np.isfinite(Pm).all()
        mask_pts.append(Pm)
        valid_mask.append(ok)

    Nc, Nm = len(clusters), len(masks)
    C = np.full((Nc, Nm), np.inf, np.float64)

    # ---- 4) fill cost matrix ----
    for i, c in enumerate(clusters):
        pts_xyz = np.asarray(c["points_xyz"], np.float64)
        if pts_xyz.size == 0:
            continue

        if mode == "xz":
            cnt_c, info = cluster_contour_auto(
                pts_xyz,
                mirror_x=mirror_x, mirror_z=mirror_z,
                grid=grid, splat_r=splat_r, close_r=close_r,
                want="Y_left",
                verbose=verbose,
                cluster_id=c["cluster_id"]
            )

            if cnt_c.shape[0] < 3 or not np.isfinite(cnt_c).all():
                continue




            for j in range(Nm):
                if not valid_mask[j]:
                    continue
                C[i, j] = shape_context_distance_rot(cnt_c, mask_pts[j], n=n_samples, n_rot=n_rot)

        elif mode == "image":
            if projector is None:
                raise ValueError("Za mode='image' moraš dati projector(points_xyz)->uv")

            uv = projector(pts_xyz)
            if uv is None or uv.shape[0] < 3:
                continue

            uv = np.asarray(uv, np.float64)
            if not np.isfinite(uv).all():
                uv = uv[np.isfinite(uv).all(axis=1)]
            if uv.shape[0] < 3:
                continue

            Pc = _convex_hull_poly(uv)
            if Pc is None or Pc.shape[0] < 3:
                continue

            for j in range(Nm):
                if not valid_mask[j]:
                    continue
                C[i, j] = shape_context_distance_rot(Pc, mask_pts[j], n=n_samples, n_rot=n_rot)

        else:
            raise ValueError("mode mora biti 'xz' ili 'image'")

    # ---- 5)  infeasible matrix for Hungarian ----
    finite_row = np.isfinite(C).any(axis=1)
    finite_col = np.isfinite(C).any(axis=0)

    if verbose:
        print("C shape:", C.shape)
        print("  all-inf rows:", int(np.sum(~finite_row)))
        print("  all-inf cols:", int(np.sum(~finite_col)))
        print("  finite entries:", int(np.sum(np.isfinite(C))))

    if not finite_row.any() or not finite_col.any():
        if verbose:
            print("No feasible pairs at all (everything inf). Check raster params / flips / masks.")
        return [], C, fd2

    C2 = C[finite_row][:, finite_col]

    if C2.size == 0 or not (np.isfinite(C2).any(axis=1).all() and np.isfinite(C2).any(axis=0).all()):
        if verbose:
            print("Still infeasible after removing empty rows/cols. Try reducing close_r/splat_r or check contours.")
        return [], C, fd2

    ri2, cj2 = linear_sum_assignment(C2)

    row_idx = np.where(finite_row)[0]
    col_idx = np.where(finite_col)[0]
    ri = row_idx[ri2]
    cj = col_idx[cj2]

    # ---- 6) build matches ----
    matches = []
    for i, j in zip(ri, cj):
        if not np.isfinite(C[i, j]):
            continue
        matches.append({
            "cluster_id": clusters[i]["cluster_id"],
            "mask_id": masks[j]["mask_id"],
            "cost": float(C[i, j]),
            "i": int(i),
            "j": int(j),
        })
    matches.sort(key=lambda d: d["cost"])

    return matches, C, fd2


def _mask_contours(mask_u8):
    mk = mask_u8.astype(np.uint8)
    if mk.max() <= 1:
        mk = mk * 255
    cnts, _ = cv2.findContours(mk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return cnts

def _convex_hull_poly(points_xy):
    P = np.asarray(points_xy, np.float64)
    if P.shape[0] < 3:
        return None
    pts = P.astype(np.float32).reshape(-1, 1, 2)
    hull = cv2.convexHull(pts)[:, 0, :]
    return hull.astype(np.float64)

def plot_cluster_mask_matches(
    fd,
    matches,
    projector=None,
    mirror_x=True, mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title=None,
    grid_vis=256, splat_r_vis=2, close_r_vis=4,
    want="Y_left",
):

    fd = filter_fd_min_points(fd, min_cluster_points=min_cluster_points)

    clu_by_id = {c["cluster_id"]: c for c in fd.get("clusters", [])}
    msk_by_id = {m["mask_id"]: m for m in fd.get("masks", [])}

    matches_ok = []
    for mm in matches:
        cid, mid = mm["cluster_id"], mm["mask_id"]
        if cid in clu_by_id and mid in msk_by_id:
            matches_ok.append(mm)

    if not matches_ok:
        print("Nema matchova nakon filtera.")
        return

    matched_cids = set(mm["cluster_id"] for mm in matches_ok)
    matched_mids = set(mm["mask_id"] for mm in matches_ok)

    img = cv2.imread(fd["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]

    fig, (ax_img, ax_lid) = plt.subplots(
        1, 2, figsize=(18, 8),
        gridspec_kw={"width_ratios": [1.25, 1.0]}
    )

    cmap = plt.get_cmap("tab10")
    colors = [cmap(i % 10) for i in range(len(matches_ok))]

   -
    ax_img.imshow(img_rgb)
    ax_img.set_xlim(0, W)
    ax_img.set_ylim(H, 0)
    ax_img.axis("off")
    ax_img.set_title(title or "Matches: masks colored")

    if show_unmatched:
        for m in fd.get("masks", []):
            if m["mask_id"] in matched_mids:
                continue
            for cnt in _mask_contours(m["mask"]):
                cnt = cnt.squeeze(1)
                if cnt.shape[0] >= 3:
                    ax_img.add_patch(Polygon(cnt, closed=True, fill=False, linewidth=1.0, alpha=0.15))

    legend_elems = []
    for k, (mm, col) in enumerate(zip(matches_ok, colors), start=1):
        cid, mid = mm["cluster_id"], mm["mask_id"]
        m = msk_by_id[mid]
        c = clu_by_id[cid]

        # mask outline
        for cnt in _mask_contours(m["mask"]):
            cnt = cnt.squeeze(1)
            if cnt.shape[0] >= 3:
                ax_img.add_patch(Polygon(cnt, closed=True, fill=False, edgecolor=col, linewidth=2.8))

        # mask centroid label
        mx, my = m["centroid_xy"]
        extra = f" | cost={mm['cost']:.2f}" if "cost" in mm else ""
        ax_img.plot(mx, my, marker="o", markersize=8, color=col, markeredgecolor="black")
        ax_img.text(
            mx + 8, my - 8, f"{k}) M{mid} ⇄ C{cid}{extra}",
            color="white", fontsize=11,
            bbox=dict(facecolor=col, alpha=0.85, edgecolor="none", boxstyle="round,pad=0.25")
        )


        if projector is not None:
            pts_xyz = np.asarray(c["points_xyz"], np.float64)
            uv = projector(pts_xyz)
            if uv is not None and uv.shape[0] >= 3:
                ok = (
                    np.isfinite(uv).all(axis=1) &
                    (uv[:,0] >= 0) & (uv[:,0] < W) &
                    (uv[:,1] >= 0) & (uv[:,1] < H)
                )
                uv_ok = uv[ok]
                hull = _convex_hull_poly(uv_ok)
                if hull is not None and hull.shape[0] >= 3:
                    ax_img.add_patch(Polygon(hull, closed=True, fill=False, edgecolor=col, linewidth=1.8, alpha=0.9))

        legend_elems.append(Line2D([0],[0], color=col, lw=4, label=f"{k}) M{mid} ⇄ C{cid}{extra}"))

    ax_img.legend(handles=legend_elems, loc="lower left", fontsize=9, framealpha=0.9)

    ax_lid.set_title("LiDAR best-plane (XY/XZ/YZ) debug view")
    ax_lid.set_xlabel("axis-1")
    ax_lid.set_ylabel("axis-2")
    ax_lid.grid(True, alpha=0.2)
    ax_lid.set_aspect("equal", adjustable="box")

    if show_unmatched:
        for c_un in fd.get("clusters", []):
            cid_un = c_un["cluster_id"]
            if cid_un in matched_cids:
                continue
            pts = np.asarray(c_un["points_xyz"], np.float64)
            if pts.size == 0:
                continue

            try:
                _, info_u, P2_u = cluster_contour_auto(
                    pts,
                    mirror_x=mirror_x, mirror_z=mirror_z,
                    grid=grid_vis, splat_r=splat_r_vis, close_r=close_r_vis,
                    want=want,
                    verbose=False,
                    cluster_id=cid_un,
                    return_points=True
                )
                ax_lid.scatter(P2_u[:,0], P2_u[:,1], s=2, alpha=0.10)
            except Exception:
                continue

    # matched clusters (colored) + centroid
    for k, (mm, col) in enumerate(zip(matches_ok, colors), start=1):
        cid = mm["cluster_id"]
        c = clu_by_id[cid]
        pts = np.asarray(c["points_xyz"], np.float64)
        if pts.size == 0:
            continue

        _, info, P2 = cluster_contour_auto(
            pts,
            mirror_x=mirror_x, mirror_z=mirror_z,
            grid=grid_vis, splat_r=splat_r_vis, close_r=close_r_vis,
            want=want,
            verbose=False,
            cluster_id=cid,
            return_points=True
        )
        ax_lid.scatter(P2[:,0], P2[:,1], s=2, alpha=0.55, color=col)

        centroid = np.asarray(c["centroid_xyz"], np.float64)
        cx = -centroid[0] if mirror_x else centroid[0]
        cy = centroid[1]  # mirror_y=False
        cz = -centroid[2] if mirror_z else centroid[2]

        plane = info["plane"]
        flip2 = info["flip2"]
        flip_u = info["flip_u"]

        if plane == "xy":
            p1, p2 = cx, cy
        elif plane == "xz":
            p1, p2 = cx, cz
        else:  # yz
            p1, p2 = cy, cz

        if flip2:
            p2 = -p2
        if flip_u:
            p1 = -p1

        ax_lid.plot(p1, p2, marker="o", markersize=9, color=col, markeredgecolor="black")
        ax_lid.text(
            p1 + 0.2, p2 + 0.2, f"{k}) C{cid}",
            fontsize=11,
            bbox=dict(facecolor="white", alpha=0.85, edgecolor="none", boxstyle="round,pad=0.2")
        )

    plt.tight_layout()
    plt.show()




def debug_one_cluster_shape(fd, cluster_id, mirror_x=True, mirror_z=False,
                           grid=256, splat_r=2, close_r=4):
    c = next((cc for cc in fd["clusters"] if cc["cluster_id"] == cluster_id), None)
    if c is None:
        print("Nema tog cluster_id")
        return

    pts = np.asarray(c["points_xyz"], np.float64)

    plane, flip2 = choose_best_plane_by_raster(pts, mirror_x=mirror_x, mirror_z=mirror_z,
                                           grid=grid, splat_r=splat_r, close_r=close_r, verbose=True)

    P2 = glue_to_plane(pts, plane=plane, mirror_x=mirror_x, mirror_y=False, mirror_z=mirror_z)

    if flip2:
        P2 = P2.copy()
        P2[:,1] *= -1.0

    P2, flip_u, corr = orient_plane_by_rule(pts, P2, plane=plane, want="Y_left")
    print(f"[DEBUG] C{cluster_id}: chosen={plane.upper()} flip2={flip2} flip_u={flip_u} corr(u,Y)={corr:.3f}")

    cnt, B = contour_from_points_raster(P2, grid=grid, splat_r=splat_r, close_r=close_r)

    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.scatter(P2[:,0], P2[:,1], s=3)
    plt.gca().set_aspect("equal","box")
    plt.title(f"raw {plane.upper()} cluster {cluster_id} (flip2={flip2})")
    plt.grid(True, alpha=0.2)

    plt.subplot(1,3,2)
    plt.imshow(B, cmap="gray"); plt.title("raster"); plt.axis("off")

    plt.subplot(1,3,3)
    plt.imshow(B, cmap="gray")
    if cnt.shape[0] > 0:
        plt.plot(cnt[:,0], cnt[:,1], linewidth=2)
    plt.title("contour"); plt.axis("off")
    plt.tight_layout(); plt.show()
#plotanje razlicitih indeksa
fd = build_frame_data_instances(25)
matches, C, fd_filt = match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True,
    mirror_z=False,
    only_y_lt_0=False,
    min_cluster_points=200,
    n_samples=80,
    grid=256,
    splat_r=5,
    close_r=12,
    n_rot=12
)

plot_cluster_mask_matches(
    fd_filt,
    matches,
    projector=None,
    mirror_x=True,
    mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title="Shape Context (raster contour + rotation search) | XZ mode"
)
fd = build_frame_data_instances(40)
matches, C, fd_filt = match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True,
    mirror_z=False,
    only_y_lt_0=False,
    min_cluster_points=200,
    n_samples=80,
    grid=256,
    splat_r=5,
    close_r=12,
    n_rot=12
)

plot_cluster_mask_matches(
    fd_filt,
    matches,
    projector=None,
    mirror_x=True,
    mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title="Shape Context (raster contour + rotation search) | XZ mode"
)
fd = build_frame_data_instances(41)

matches, C, fd_filt = match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True,
    mirror_z=False,
    only_y_lt_0=False,
    min_cluster_points=200,
    n_samples=80,
    grid=256,
    splat_r=5,
    close_r=12,
    n_rot=12
)

plot_cluster_mask_matches(
    fd_filt,
    matches,
    projector=None,
    mirror_x=True,
    mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title="Shape Context (raster contour + rotation search) | XZ mode"
)
fd = build_frame_data_instances(42)

matches, C, fd_filt = match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True,
    mirror_z=False,
    only_y_lt_0=False,
    min_cluster_points=200,
    n_samples=80,
    grid=256,
    splat_r=5,
    close_r=12,
    n_rot=12
)

plot_cluster_mask_matches(
    fd_filt,
    matches,
    projector=None,
    mirror_x=True,
    mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title="Shape Context (raster contour + rotation search) | XZ mode"
)
fd = build_frame_data_instances(43)

matches, C, fd_filt = match_clusters_masks_shape_context(
    fd,
    mode="xz",
    mirror_x=True,
    mirror_z=False,
    only_y_lt_0=False,
    min_cluster_points=200,
    n_samples=80,
    grid=256,
    splat_r=5,
    close_r=12,
    n_rot=12
)

plot_cluster_mask_matches(
    fd_filt,
    matches,
    projector=None,
    mirror_x=True,
    mirror_z=False,
    min_cluster_points=200,
    show_unmatched=True,
    title="Shape Context (raster contour + rotation search) | XZ mode"
)



In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from scipy.optimize import minimize

def make_A_lidar_to_cam0(mirror_x=True):
    return np.array([
        [(-1.0 if mirror_x else 1.0), 0.0,  0.0],
        [0.0,  0.0, -1.0],
        [0.0, -1.0,  0.0],
    ], dtype=np.float64)

def lidar_to_cam0(points_xyz, A):
    P = np.asarray(points_xyz, dtype=np.float64)
    return (A @ P.T).T

def mask_contours(mask_u8):
    mk = mask_u8.astype(np.uint8)
    if mk.max() == 1: mk = mk * 255
    cnts, _ = cv2.findContours(mk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return cnts

def mask_dt(mask_u8):
    mk = mask_u8.astype(np.uint8)
    if mk.max() == 1: mk = mk * 255
    dt = cv2.distanceTransform((mk == 0).astype(np.uint8), cv2.DIST_L2, 5).astype(np.float64)
    return dt

def bilinear_sample(img, u, v):
    """img: (H,W) float64, u,v: (N,) float coords"""
    H, W = img.shape
    u = np.asarray(u, np.float64)
    v = np.asarray(v, np.float64)

    u0 = np.floor(u).astype(np.int32)
    v0 = np.floor(v).astype(np.int32)
    u1 = u0 + 1
    v1 = v0 + 1

    u0c = np.clip(u0, 0, W-1); u1c = np.clip(u1, 0, W-1)
    v0c = np.clip(v0, 0, H-1); v1c = np.clip(v1, 0, H-1)

    fu = u - u0
    fv = v - v0

    Ia = img[v0c, u0c]
    Ib = img[v0c, u1c]
    Ic = img[v1c, u0c]
    Id = img[v1c, u1c]

    wa = (1-fu)*(1-fv)
    wb = fu*(1-fv)
    wc = (1-fu)*fv
    wd = fu*fv

    return wa*Ia + wb*Ib + wc*Ic + wd*Id

def project_cam0_to_image(P0, rvec, tvec, K, dist):
    rvec = np.asarray(rvec, np.float64).reshape(3,1)
    tvec = np.asarray(tvec, np.float64).reshape(3,1)
    dist = np.asarray(dist, np.float64).reshape(-1,1)

    R, _ = cv2.Rodrigues(rvec)
    Pc = (R @ P0.T + tvec).T
    Z = Pc[:,2]

    uv, _ = cv2.projectPoints(P0.reshape(-1,1,3), rvec, tvec, K, dist)
    uv = uv.reshape(-1,2)
    return uv, Z

def overlay(fd, cluster, mask, uv, title=""):
    img = cv2.imread(fd["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]

    ok = np.isfinite(uv).all(axis=1) & (uv[:,0]>=0) & (uv[:,0]<W) & (uv[:,1]>=0) & (uv[:,1]<H)

    fig, ax = plt.subplots(figsize=(14,8))
    ax.imshow(img_rgb)
    ax.set_xlim(0, W); ax.set_ylim(H, 0); ax.axis("off")

    for cnt in mask_contours(mask["mask"]):
        cnt = cnt.squeeze(1)
        if cnt.shape[0] >= 3:
            ax.add_patch(Polygon(cnt, closed=True, fill=False, edgecolor="lime", linewidth=3))

    ax.scatter(uv[ok,0], uv[ok,1], s=10, alpha=0.8)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()
def soft_point_mask(u, v, H, W):
    """
    u,v: float coords (N,), već filtrirani da su u [0,W-2], v [0,H-2]
    return: (H,W) float64 u [0,1] approx
    """
    p = np.zeros((H, W), np.float64)

    u0 = np.floor(u).astype(np.int32)
    v0 = np.floor(v).astype(np.int32)
    du = u - u0
    dv = v - v0

    w00 = (1-du)*(1-dv)
    w10 = du*(1-dv)
    w01 = (1-du)*dv
    w11 = du*dv

    np.add.at(p, (v0,     u0    ), w00)
    np.add.at(p, (v0,     u0 + 1), w10)
    np.add.at(p, (v0 + 1, u0    ), w01)
    np.add.at(p, (v0 + 1, u0 + 1), w11)

    p = np.clip(p, 0.0, 1.0)
    return p

def soft_iou(point_prob, mask01, eps=1e-9):
    inter = np.sum(point_prob * mask01)
    union = np.sum(point_prob + mask01 - point_prob * mask01)
    return inter / (union + eps)


def refine_c5_to_mask_powell(frame_idx=0, cluster_id=5, manual_matches=None, mirror_x=True, stride=1,
                             max_points=600, seed=0,
                             w_out=50.0, w_back=80.0, w_reg=0.02,
                             init=None):



    fd = build_frame_data_instances(frame_idx)

    if manual_matches is None or cluster_id not in manual_matches:
        raise ValueError(f"Nema ručnog matcha za cluster C{cluster_id}. Daj manual_matches={{cluster_id: mask_id}}")
    mask_id = manual_matches[cluster_id]

    clusters_by_id = {c["cluster_id"]: c for c in fd["clusters"]}
    masks_by_id    = {m["mask_id"]: m for m in fd["masks"]}

    cluster = clusters_by_id[cluster_id]
    mask    = masks_by_id[mask_id]

    pts = np.asarray(cluster["points_xyz"], np.float64)[::max(1,stride)]

    med = np.median(np.abs(pts[:,1]))
    if med > 500:
        pts = pts / 1000.0
        print("Detected mm -> converted to meters.")
    else:
        print("Detected meters -> kept as-is.")

    rng = np.random.default_rng(seed)
    if pts.shape[0] > max_points:
        idx = rng.choice(pts.shape[0], size=max_points, replace=False)
        pts = pts[idx]

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    P0 = lidar_to_cam0(pts, A)

    img = cv2.imread(fd["image_path"])
    H, W = img.shape[:2]
    mask01 = (mask["mask"].astype(np.uint8) > 0).astype(np.float64)


    if init is None:
        x0 = np.zeros(6, np.float64)
    else:
        x0 = np.array(init, np.float64).copy()

    """
    def cost(x):
        rvec = x[:3]
        tvec = x[3:6]
        uv, Z = project_cam0_to_image(P0, rvec, tvec, K, dist)

        u = uv[:,0]; v = uv[:,1]
        finite = np.isfinite(u) & np.isfinite(v)

        inside = finite & (u >= 0) & (u < (W-1)) & (v >= 0) & (v < (H-1))
        front  = Z > 1e-3
        good = inside & front

        out_pen  = w_out  * (1.0 - float(np.mean(inside)))
        back_pen = w_back * (1.0 - float(np.mean(front)))
        reg      = w_reg  * float(np.sum(x*x))

        if not np.any(good):
            return 1.0 + out_pen + back_pen + reg

        mvals = bilinear_sample(mask01, u[good], v[good])  # u [0,1]
        inlier = float(np.mean(mvals))                     # 1.0 je idealno

        return (1.0 - inlier) + out_pen + back_pen + reg
    """
    def cost(x):
      rvec = x[:3]
      tvec = x[3:6]
      uv, Z = project_cam0_to_image(P0, rvec, tvec, K, dist)

      u = uv[:,0]; v = uv[:,1]
      finite = np.isfinite(u) & np.isfinite(v)

      inside = finite & (u >= 0) & (u < (W-1)) & (v >= 0) & (v < (H-1))
      front  = Z > 1e-3
      good = inside & front

      out_pen  = w_out  * (1.0 - float(np.mean(inside)))
      back_pen = w_back * (1.0 - float(np.mean(front)))
      reg      = w_reg  * float(np.sum(x*x))

      if not np.any(good):
          return 1.0 + out_pen + back_pen + reg  # total fail

      # soft point-mask + IoU
      p = soft_point_mask(u[good], v[good], H, W)
      iou = soft_iou(p, mask01)

      # minimizira se
      return (1.0 - iou) + out_pen + back_pen + reg








    uv0, _ = project_cam0_to_image(P0, x0[:3], x0[3:], K, dist)
    print(f"C0 cost: {cost(x0):.3f}")
    overlay(fd, cluster, mask, uv0, title=f"BEFORE: C{cluster_id}→M{mask_id} (x0)")

    #res = minimize(cost, x0, method="Powell",
      #              options=dict(maxiter=200, disp=True))

    res = minimize(cost, x0, method="Nelder-Mead",
               options=dict(maxiter=2000, xatol=1e-6, fatol=1e-6, disp=True))


    print("success:", res.success)
    print("message:", res.message)
    print("nit:", res.nit)
    print("nfev:", res.nfev)
    print("final cost:", res.fun)


    x_opt = res.x
    uv1, _ = project_cam0_to_image(P0, x_opt[:3], x_opt[3:], K, dist)
    print(f"C1 cost: {cost(x_opt):.3f}")
    print("x_opt:", x_opt)

    overlay(fd, cluster, mask, uv1, title=f"AFTER: C{cluster_id}→M{mask_id} ")

    return {
        "frame_idx": frame_idx,
        "cluster_id": cluster_id,
        "mask_id": mask_id,
        "x0": x0,
        "x_opt": x_opt,
        "minimize_result": res
    }


init = [0,      1.5708,           0, 0,0,0]
manual_matches = {4 : 0}   # cluster_id -> mask_id
out = refine_c5_to_mask_powell(frame_idx=25, cluster_id=4, manual_matches=manual_matches, mirror_x=True, stride=1, init=init)

#out = refine_c5_to_mask_powell(frame_idx=50, cluster_id=1, manual_matches=manual_matches, mirror_x=True, stride=1, init=init)
#out = refine_c5_to_mask_powell(frame_idx=55, cluster_id=1, manual_matches=manual_matches, mirror_x=True, stride=1, init=init)


In [ ]:
def overlay_raw_lidar_after_calib(frame_idx, rvec, tvec=np.zeros(3), mirror_x=True,
                                 stride=1, max_points=30000, seed=0,
                                 show_masks=False, title="RAW LiDAR overlay"):
    fd = build_frame_data_instances(frame_idx)
    img = cv2.imread(fd["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    H, W = img.shape[:2]

    pts = load_kitti_bin(fd["lidar_path"])[:, :3].astype(np.float64)[::max(1,stride)]


    rng = np.random.default_rng(seed)
    if pts.shape[0] > max_points:
        idx = rng.choice(pts.shape[0], size=max_points, replace=False)
        pts = pts[idx]

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    P0 = lidar_to_cam0(pts, A)

    uv, Z = project_cam0_to_image(P0, rvec, tvec, K, dist)

    ok = (
        np.isfinite(uv).all(axis=1) &
        (uv[:,0] >= 0) & (uv[:,0] < W) &
        (uv[:,1] >= 0) & (uv[:,1] < H) &
        (Z > 1e-3)
    )

    print("Z stats:", np.min(Z), np.median(Z), np.max(Z), " ok:", int(ok.sum()), "/", len(ok))

    fig, ax = plt.subplots(figsize=(14,8))
    ax.imshow(img_rgb); ax.set_xlim(0,W); ax.set_ylim(H,0); ax.axis("off")

    if show_masks:
        for m in fd.get("masks", []):
            for cnt in mask_contours(m["mask"]):
                cnt = cnt.squeeze(1)
                if cnt.shape[0] >= 3:
                    ax.add_patch(Polygon(cnt, closed=True, fill=False, linewidth=1.0, alpha=0.25))

    ax.scatter(uv[ok,0], uv[ok,1], s=2, alpha=0.5)
    ax.set_title(f"{title} | plotted={int(ok.sum())}/{len(ok)}")
    plt.tight_layout(); plt.show()
print(out["x_opt"])
"""
overlay_raw_lidar_after_calib(
    frame_idx=45,
    rvec=out["x_opt"][:3],
    tvec=out["x_opt"][3:],
    mirror_x=True,
    show_masks=True
)
"""

#out["x_opt"] = [-0.00040471,      1.5957,   0.0013399 , -0.0017503,   0.0022085, -0.00070122]
#eval_and_plot_one(frame_idx=40, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=41, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=42, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=43, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
print("ZADAN X OPTTTTTTTT ###################################################################################################################################################################")
print("###################################################################################################################################################################################")
#out["x_opt"] = [0,      1.5708,   0.017,  0,   0, 0]
#eval_and_plot_one(frame_idx=40, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=41, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=42, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
#eval_and_plot_one(frame_idx=43, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
eval_and_plot_one(frame_idx=50, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
eval_and_plot_one(frame_idx=52, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
eval_and_plot_one(frame_idx=47, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)
eval_and_plot_one(frame_idx=48, rvec= out["x_opt"][:3], tvec=out["x_opt"][3:], mirror_x=True)








In [ ]:
import numpy as np
import cv2
import pandas as pd

def rotation_geodesic_deg(rvec_est, rvec_gt):
    R_est, _ = cv2.Rodrigues(np.asarray(rvec_est, np.float64).reshape(3,1))
    R_gt,  _ = cv2.Rodrigues(np.asarray(rvec_gt,  np.float64).reshape(3,1))
    R_rel = R_est @ R_gt.T
    tr = np.trace(R_rel)
    cosang = (tr - 1.0) / 2.0
    cosang = float(np.clip(cosang, -1.0, 1.0))
    ang = np.arccos(cosang)
    return float(np.rad2deg(ang))

def translation_l2(t_est, t_gt):
    t_est = np.asarray(t_est, np.float64).ravel()
    t_gt  = np.asarray(t_gt,  np.float64).ravel()
    return float(np.linalg.norm(t_est - t_gt))

def eval_one_frame_vs_gt(frame_idx, x_est, x_gt, mirror_x=True,
                         stride=2, max_points=120000, seed=0,
                         canny_low=80, canny_high=160, depth_grad_thr=0.35,
                         tau_px=3.0):
    fd = build_frame_data_instances(frame_idx)
    img = cv2.imread(fd["image_path"])
    H, W = img.shape[:2]

    # points
    pts = load_kitti_bin(fd["lidar_path"])[:, :3].astype(np.float64)
    if stride > 1:
        pts = pts[::stride]
    if (max_points is not None) and (pts.shape[0] > max_points):
        rng = np.random.default_rng(seed)
        idx = rng.choice(pts.shape[0], size=max_points, replace=False)
        pts = pts[idx]

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    P0 = lidar_to_cam0(pts, A)

    r_est, t_est = np.asarray(x_est[:3]), np.asarray(x_est[3:])
    r_gt,  t_gt  = np.asarray(x_gt[:3]),  np.asarray(x_gt[3:])

    #   extrinsics error (global, ne ovisi o frameu)
    rot_err_deg = rotation_geodesic_deg(r_est, r_gt)
    trans_err_m = translation_l2(t_est, t_gt)

    # reprojection pixel error
    uv_e, Z_e = project_cam0_to_image(P0, r_est, t_est, K, dist)
    uv_g, Z_g = project_cam0_to_image(P0, r_gt,  t_gt,  K, dist)

    ue, ve = uv_e[:,0], uv_e[:,1]
    ug, vg = uv_g[:,0], uv_g[:,1]

    finite = np.isfinite(ue) & np.isfinite(ve) & np.isfinite(ug) & np.isfinite(vg)
    inside_e = (ue >= 0) & (ue < W) & (ve >= 0) & (ve < H)
    inside_g = (ug >= 0) & (ug < W) & (vg >= 0) & (vg < H)
    front_e  = Z_e > 1e-3
    front_g  = Z_g > 1e-3

    good = finite & inside_e & inside_g & front_e & front_g

    if np.any(good):
        du = ue[good] - ug[good]
        dv = ve[good] - vg[good]
        reproj_px = np.sqrt(du*du + dv*dv)
        reproj_mean = float(np.mean(reproj_px))
        reproj_med  = float(np.median(reproj_px))
        reproj_p95  = float(np.percentile(reproj_px, 95))
        good_ratio  = float(np.mean(good))
        n_good = int(np.sum(good))
    else:
        reproj_mean = reproj_med = reproj_p95 = 1e9
        good_ratio = 0.0
        n_good = 0

    #  edge alignment metrics
    depth_e = depth_image_from_points(P0, r_est, t_est, K, dist, H, W)
    e_img = image_edges(img, low=canny_low, high=canny_high)
    dt_img = distance_transform_of_edges(e_img)
    e_depth, _ = depth_edges(depth_e, grad_thr=depth_grad_thr)

    edge_dt_mean, info = edge_alignment_cost(dt_img, e_depth)
    depth_edge_pixels = int(info["depth_edge_pixels"])

    # precision@tau: koliko depth-edge pixela ima DT <= tau
    ys, xs = np.where(e_depth > 0)
    if len(xs) > 0 and np.isfinite(edge_dt_mean):
        edge_prec_tau = float(np.mean(dt_img[ys, xs] <= tau_px))
    else:
        edge_prec_tau = 0.0

    return dict(
        frame=frame_idx,
        rot_err_deg=rot_err_deg,
        trans_err_m=trans_err_m,
        reproj_mean_px=reproj_mean,
        reproj_median_px=reproj_med,
        reproj_p95_px=reproj_p95,
        n_good=n_good,
        good_ratio=good_ratio,
        edge_dt_mean=edge_dt_mean,
        depth_edge_pixels=depth_edge_pixels,
        edge_prec_tau=edge_prec_tau
    )

def eval_sequence_vs_gt(frames, x_est, x_gt, **kwargs):
    rows = [eval_one_frame_vs_gt(fr, x_est, x_gt, **kwargs) for fr in frames]
    df = pd.DataFrame(rows)
    summary = df.drop(columns=["frame"]).agg(["mean","std","min","max"])
    return df, summary


In [ ]:
frames = [i for i in range(40, 56)]
x_est = [-0.00040471,      1.5957,   0.0013399 , -0.0017503,   0.0022085, -0.00070122]
#x_est = [ 0,      1.4835  ,         0, 0,0,0]
x_gt  = [0,      1.5708,   0.017,  0,   0, 0]  # ti ubaci svoj GT 6D vektor [rvec(3), tvec(3)]

df, summary = eval_sequence_vs_gt(
    frames, x_est, x_gt,
    mirror_x=True,
    stride=2,
    max_points=120000,
    canny_low=80, canny_high=160,
    depth_grad_thr=0.35,
    tau_px=3.0
)

print(df)
print(summary)


In [ ]:
def visualize_edge_alignment(frame_idx, rvec, tvec, mirror_x=True,
                             canny_low=80, canny_high=160,
                             depth_grad_thr=0.35, tau_px=3.0):
    fd = build_frame_data_instances(frame_idx)
    img = cv2.imread(fd["image_path"])
    H, W = img.shape[:2]

    pts = load_kitti_bin(fd["lidar_path"])[:, :3]
    A = make_A_lidar_to_cam0(mirror_x=mirror_x)
    P0 = lidar_to_cam0(pts, A)

    depth = depth_image_from_points(P0, rvec, tvec, K, dist, H, W)

    e_img = image_edges(img, low=canny_low, high=canny_high)
    dt_img = distance_transform_of_edges(e_img)
    e_depth, _ = depth_edges(depth, grad_thr=depth_grad_thr)

    ys, xs = np.where(e_depth > 0)
    n = len(xs)
    if n == 0:
        print("No depth edge pixels.")
        return

    vals = dt_img[ys, xs]
    edge_dt_mean = float(np.mean(vals))
    edge_prec_tau = float(np.mean(vals <= tau_px))

    out = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()

    out[e_img > 0] = [0, 255, 0]

    close = vals <= tau_px
    out[ys[close], xs[close]] = [0, 255, 255]
    out[ys[~close], xs[~close]] = [255, 0, 0]

    plt.figure(figsize=(14,7))
    plt.imshow(out)
    plt.title(
        f"Frame {frame_idx} | meanDT={edge_dt_mean:.3f}px | prec@{tau_px}px={edge_prec_tau:.3f} | depthEdgePix={n}\n"
        f"green=image edges, cyan=depth edges <= {tau_px}px, red=depth edges > {tau_px}px"
    )
    plt.axis("off")
    plt.show()

    plt.figure(figsize=(8,4))
    plt.hist(vals, bins=60)
    plt.axvline(tau_px, linewidth=2)
    plt.title(f"DT distribution on depth-edge pixels (frame {frame_idx})")
    plt.xlabel("distance to nearest image edge (px)")
    plt.ylabel("count")
    plt.show()

    return edge_dt_mean, edge_prec_tau, n
x = [-0.00040471,      1.5957,   0.0013399 , -0.0017503,   0.0022085, -0.00070122]
#x = [ 0,      1.4835  ,         0, 0,0,0]

visualize_edge_alignment(40, x[:3], x[3:], mirror_x=True, tau_px=3.0)
visualize_edge_alignment(41, x[:3], x[3:], mirror_x=True, tau_px=3.0)
visualize_edge_alignment(42, x[:3], x[3:], mirror_x=True, tau_px=3.0)
visualize_edge_alignment(43, x[:3], x[3:], mirror_x=True, tau_px=3.0)


In [ ]:
import numpy as np
import pandas as pd
import cv2

def cluster_mask_metrics_for_frame(
    frame_idx,
    rvec, tvec,
    mirror_x=True,
    stride=1,
    max_points_per_cluster=2000,
    seed=0,
    min_projected_points=50,   # klaster je "vidljiv" ako ima >= ovoliko good točaka
    min_inlier_ratio=0.10,     # klaster "u maski" ako best_ratio >= ovo
    min_inliers=30,            # i best_inliers >= ovo
    z_min=1e-3                 # ispred kamere
):
    fd = build_frame_data_instances(frame_idx)
    img = cv2.imread(fd["image_path"])
    H, W = img.shape[:2]

    # masks (union + list)
    masks = fd["masks"]
    if len(masks) == 0:
        raise ValueError(f"Frame {frame_idx}: nema maski u fd['masks'].")

    mask_u8_list = []
    for m in masks:
        mk = m["mask"].astype(np.uint8)
        if mk.shape[:2] != (H, W):
            mk = cv2.resize(mk, (W, H), interpolation=cv2.INTER_NEAREST)
        mask_u8_list.append(mk)

    union_mask = np.zeros((H, W), np.uint8)
    for mk in mask_u8_list:
        union_mask = cv2.bitwise_or(union_mask, mk)

    A = make_A_lidar_to_cam0(mirror_x=mirror_x)

    rng = np.random.default_rng(seed)
    rows = []

    for c in fd["clusters"]:
        cid = c["cluster_id"]
        pts = np.asarray(c["points_xyz"], np.float64)

        if stride > 1:
            pts = pts[::stride]

        if (max_points_per_cluster is not None) and (pts.shape[0] > max_points_per_cluster):
            idx = rng.choice(pts.shape[0], size=max_points_per_cluster, replace=False)
            pts = pts[idx]

        P0 = lidar_to_cam0(pts, A)

        uv, Z = project_cam0_to_image(P0, rvec, tvec, K, dist)
        u = uv[:, 0]
        v = uv[:, 1]

        finite = np.isfinite(u) & np.isfinite(v)
        inside = finite & (u >= 0) & (u < W) & (v >= 0) & (v < H)
        front  = Z > z_min
        good = inside & front

        n_total = int(len(u))
        n_good = int(np.sum(good))

        if n_good < min_projected_points:
            rows.append(dict(
                frame=frame_idx, cluster_id=cid,
                n_points_3d=n_total, n_projected=n_good,
                visible=False,
                best_mask_id=None,
                best_inliers=0,
                best_inlier_ratio=0.0,
                in_any_mask_ratio=0.0,
                matched=False
            ))
            continue

        u_i = u[good].astype(np.int32)
        v_i = v[good].astype(np.int32)

        # koliko upada u barem neku masku
        in_any = (union_mask[v_i, u_i] > 0)
        in_any_cnt = int(np.sum(in_any))
        in_any_ratio = float(in_any_cnt / n_good)

        # best mask per cluster
        best_mask_id = None
        best_inliers = -1
        for mi, mk in enumerate(mask_u8_list):
            cnt = int(np.sum(mk[v_i, u_i] > 0))
            if cnt > best_inliers:
                best_inliers = cnt
                best_mask_id = masks[mi]["mask_id"]

        best_ratio = float(best_inliers / n_good)

        matched = (best_inliers >= min_inliers) and (best_ratio >= min_inlier_ratio)

        rows.append(dict(
            frame=frame_idx, cluster_id=cid,
            n_points_3d=n_total, n_projected=n_good,
            visible=True,
            best_mask_id=int(best_mask_id) if best_mask_id is not None else None,
            best_inliers=int(best_inliers),
            best_inlier_ratio=best_ratio,
            in_any_mask_ratio=in_any_ratio,
            matched=bool(matched)
        ))

    df = pd.DataFrame(rows)

    visible_df = df[df["visible"]]
    clusters_visible = int(len(visible_df))
    clusters_matched = int(np.sum(visible_df["matched"])) if clusters_visible > 0 else 0
    match_rate = float(clusters_matched / clusters_visible) if clusters_visible > 0 else 0.0

    if clusters_visible > 0:
        total_proj_points = float(np.sum(visible_df["n_projected"]))
        total_in_any = float(np.sum(visible_df["in_any_mask_ratio"] * visible_df["n_projected"]))
        point_in_any_mask_ratio = total_in_any / max(1.0, total_proj_points)
        avg_best_inlier_ratio = float(np.mean(visible_df["best_inlier_ratio"]))
        avg_in_any_ratio = float(np.mean(visible_df["in_any_mask_ratio"]))
    else:
        point_in_any_mask_ratio = 0.0
        avg_best_inlier_ratio = 0.0
        avg_in_any_ratio = 0.0

    summary = dict(
        frame=frame_idx,
        clusters_total=int(len(df)),
        clusters_visible=clusters_visible,
        clusters_matched=clusters_matched,
        match_rate=match_rate,
        point_in_any_mask_ratio=float(point_in_any_mask_ratio),
        avg_best_inlier_ratio=float(avg_best_inlier_ratio),
        avg_in_any_mask_ratio=float(avg_in_any_ratio),
        thresholds=dict(min_projected_points=min_projected_points,
                        min_inlier_ratio=min_inlier_ratio,
                        min_inliers=min_inliers)
    )

    return df, summary


def cluster_mask_metrics_over_frames(frames, rvec, tvec, **kwargs):
    all_df = []
    summaries = []
    for fr in frames:
        df, summ = cluster_mask_metrics_for_frame(fr, rvec, tvec, **kwargs)
        all_df.append(df)
        summaries.append(summ)
    return pd.concat(all_df, ignore_index=True), pd.DataFrame(summaries)
x = [-0.00040471,      1.5957,   0.0013399 , -0.0017503,   0.0022085, -0.00070122]
frames = [40, 41, 42, 43]

df_clusters, df_summary = cluster_mask_metrics_over_frames(
    frames,
    rvec=x[:3], tvec=x[3:],
    mirror_x=True,
    stride=1,
    max_points_per_cluster=2000,
    min_projected_points=50,
    min_inlier_ratio=0.10,
    min_inliers=30
)

print(df_summary)
print(df_clusters.sort_values(["frame","best_inlier_ratio"], ascending=[True,False]).head(20))
